In [1]:
import MetaTrader5 as mt5
import time
from datetime import datetime, timedelta
import argparse

# Function to parse command-line arguments
def parse_arguments():
    parser = argparse.ArgumentParser(description='Forex trading bot with dynamic input parameters.')
    parser.add_argument('--symbols', type=str, nargs='+', required=True, help='List of symbols to trade')
    parser.add_argument('--times', type=str, nargs='+', required=True, help='List of scheduled times (YYYY-MM-DD HH:MM:SS)')
    parser.add_argument('--lot', type=float, required=True, help='Lot size')
    parser.add_argument('--stop_loss_pips', type=int, required=True, help='Stop loss in pips')
    parser.add_argument('--trigger_pips', type=int, required=True, help='Trigger pips for trailing stop')
    parser.add_argument('--trailing_pips', type=int, required=True, help='Trailing stop pips')
    parser.add_argument('--stop_pips', type=int, required=True, help='Stop pips for pending orders')
    parser.add_argument('--login', type=int, required=True, help='Account number')
    parser.add_argument('--password', type=str, required=True, help='Account password')
    parser.add_argument('--server', type=str, required=True, help='Broker server')

    return parser.parse_args()

# Function to convert string times to datetime objects
def convert_times(times):
    return [datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S') for time_str in times]

# Parse arguments
args = parse_arguments()

# Initialize the MetaTrader 5 platform
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    exit()

# Login to the trading account
if not mt5.login(args.login, args.password, args.server):
    print(f"Failed to connect to account {args.login}")
    mt5.shutdown()
    exit()

# Ensure AutoTrading is enabled
account_info = mt5.account_info()
if account_info is None:
    print("Failed to get account info, AutoTrading might be disabled")
    mt5.shutdown()
    exit()
elif not account_info.trade_allowed:
    print("AutoTrading is disabled, please enable it in the MetaTrader 5 platform")
    mt5.shutdown()
    exit()

# Set parameters
symbols = args.symbols
scheduled_times = convert_times(args.times)
lot = args.lot

# Explicitly define pip values for each symbol
pip_dict = {"EURUSD": 0.0001, "GBPUSD": 0.0001, "USDJPY": 0.01, "XAUUSD": 0.1}

stop_loss_pips = args.stop_loss_pips
trigger_pips = args.trigger_pips
trailing_pips = args.trailing_pips
stop_pips = args.stop_pips

# Function to ensure the symbol is available
def ensure_symbol(symbol):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Symbol {symbol} not found, cannot call order_check()")
        return False
    elif not symbol_info.visible:
        print(f"Symbol {symbol} is not visible, trying to switch on")
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available for trading")
            return False
    return True

# Function to place the orders
def place_orders(symbol):
    if not ensure_symbol(symbol):
        return

    pip = pip_dict[symbol]
    symbol_tick = mt5.symbol_info_tick(symbol)
    ask_price = symbol_tick.ask
    bid_price = symbol_tick.bid

    stop_loss_price_buy = ask_price - stop_loss_pips * pip
    stop_price_buy = ask_price + stop_pips * pip

    stop_loss_price_sell = bid_price + stop_loss_pips * pip
    stop_price_sell = bid_price - stop_pips * pip

    buy_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": stop_price_buy,
        "sl": stop_loss_price_buy,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script buy stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    sell_stop_request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": stop_price_sell,
        "sl": stop_loss_price_sell,
        "deviation": 10,
        "magic": 234000,
        "comment": "python script sell stop",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,  # Using FOK filling mode
    }

    result_buy_stop = mt5.order_send(buy_stop_request)
    if result_buy_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Buy stop order for {symbol} failed, retcode={result_buy_stop.retcode}")
        print(f"Error: {result_buy_stop}")
    else:
        print(f"Buy stop order for {symbol} placed successfully, ticket={result_buy_stop.order}")

    result_sell_stop = mt5.order_send(sell_stop_request)
    if result_sell_stop.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Sell stop order for {symbol} failed, retcode={result_sell_stop.retcode}")
        print(f"Error: {result_sell_stop}")
    else:
        print(f"Sell stop order for {symbol} placed successfully, ticket={result_sell_stop.order}")

# Function to close pending orders not executed within 3 minutes
def close_pending_orders(symbol, after_time):
    close_time = after_time + timedelta(minutes=3)
    while datetime.now() < close_time:
        time.sleep(1)

    orders = mt5.orders_get(symbol=symbol)
    if orders is None:
        print(f"No orders to close for {symbol}.")
        return

    for order in orders:
        if order.type in [mt5.ORDER_TYPE_BUY_STOP, mt5.ORDER_TYPE_SELL_STOP]:
            close_request = {
                "action": mt5.TRADE_ACTION_REMOVE,
                "order": order.ticket,
                "symbol": order.symbol,
                "volume": order.volume_current,
                "magic": order.magic,
                "comment": "python script cancel pending",
            }
            result = mt5.order_send(close_request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print(f"Failed to close order {order.ticket} for {symbol}, retcode={result.retcode}")
            else:
                print(f"Order {order.ticket} for {symbol} closed successfully.")

# Function to display the countdown timer
def display_countdown(target_time):
    while datetime.now() < target_time:
        now = datetime.now()
        remaining_time = target_time - now
        hours, remainder = divmod(remaining_time.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Time remaining: {remaining_time.days} days, {hours:02}:{minutes:02}:{seconds:02}", end="\r")
        time.sleep(1)

# Function to monitor and adjust trailing stops
def monitor_trailing_stops(symbol, target_time):
    pip = pip_dict[symbol]
    trigger_price_buy = None
    trigger_price_sell = None

    while datetime.now() < target_time + timedelta(minutes=30):
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY:
                    if trigger_price_buy is None:
                        trigger_price_buy = pos.price_open + trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).ask > trigger_price_buy:
                        new_sl = mt5.symbol_info_tick(symbol).ask - trailing_pips * pip
                        if new_sl > pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

                elif pos.type == mt5.ORDER_TYPE_SELL:
                    if trigger_price_sell is None:
                        trigger_price_sell = pos.price_open - trigger_pips * pip
                    if mt5.symbol_info_tick(symbol).bid < trigger_price_sell:
                        new_sl = mt5.symbol_info_tick(symbol).bid + trailing_pips * pip
                        if new_sl < pos.sl:
                            request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(request)
                            if result.retcode != mt5.TRADE_RETCODE_DONE:
                                print(f"Failed to modify SL for {symbol}, retcode={result.retcode}")
                            else:
                                print(f"Modified SL for {symbol} position {pos.ticket}")

        time.sleep(1)

# Function to manage multiple timers for multiple symbols
def manage_timers(symbols, scheduled_times):
    for scheduled_time in scheduled_times:
        # Display the countdown timer
        display_countdown(scheduled_time)

        # Place the orders for each symbol
        for symbol in symbols:
            place_orders(symbol)

        # Close pending orders not taken after 3 minutes for each symbol
        for symbol in symbols:
            close_pending_orders(symbol, scheduled_time)

        # Monitor trailing stops for each symbol
        for symbol in symbols:
            monitor_trailing_stops(symbol, scheduled_time)

# Manage multiple timers for multiple symbols
manage_timers(symbols, scheduled_times)

# Shutdown the MetaTrader 5 platform
mt5.shutdown()


### python script_name.py --symbols EURUSD GBPUSD USDJPY XAUUSD --times "2024-06-18 13:28:00" "2024-06-18 13:39:00" "2024-06-18 13:45:00" --lot 1.0 --stop_loss_pips 10 --trigger_pips 20 --trailing_pips 15 --stop_pips 10 --login 48438229 --password "Mq3p8!tJ" --server "HFMarketsGlobal-Demo"



usage: ipykernel_launcher.py [-h] --symbols SYMBOLS [SYMBOLS ...] --times TIMES [TIMES ...] --lot LOT --stop_loss_pips
                             STOP_LOSS_PIPS --trigger_pips TRIGGER_PIPS --trailing_pips TRAILING_PIPS --stop_pips
                             STOP_PIPS --login LOGIN --password PASSWORD --server SERVER
ipykernel_launcher.py: error: the following arguments are required: --symbols, --times, --lot, --stop_loss_pips, --trigger_pips, --trailing_pips, --stop_pips, --login, --password, --server


SystemExit: 2

C:\Users\NOC\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
